In [ ]:
import numpy as np
import pandas as pd
import pickle

from plot_utils import plot_res, plot_res_grouped
from red_utils import split_rel_results

In [ ]:
rel_df = 'rel_df.2458098.43869.ee.pkl'

In [ ]:
with open(rel_df.rsplit('.', 1)[0] + '.md.pkl', 'rb') as f:
    md = pickle.load(f)
    
df = pd.read_pickle(rel_df)
df

In [ ]:
# check results for a given frequency & time integration
test_freq = 300
test_tint = 35

resx = df.loc[(test_freq, test_tint)][5:].values.astype(float)
test_vis, test_gains = split_rel_results(resx, md['no_unq_bls'])
test_amps = np.abs(test_gains)
test_phases = np.angle(test_gains)
mean_test_amps = np.mean(test_amps)
mean_test_phases = np.mean(test_phases)

print('Gain amplitude solutions for frequency channel {} and time integration {} are:\n{}\n'\
      .format(test_freq, test_tint, test_amps))
print('Gain phase solutions are:\n{}\n'\
      .format(test_phases))
print('Gain amplitude average is: {}'.format(mean_test_amps))
print('Gain phase average is: {}'.format(mean_test_phases))

In [ ]:
# success rate percentage
success_pct = df['success'].sum() / df['success'].size * 100
print('{}% of iterations (freq/tint slice) were succesful'.format(round(success_pct, 2)))

### Number of iterations required

In [ ]:
plot_res(df, 'nit', ylim=[0, 1000])

In [ ]:
plot_res_grouped(df, 'nit', logy=True)

### Log-likelihood

In [ ]:
plot_res(df, 'fun', ylim=[-0.1, 1])

In [ ]:
plot_res_grouped(df, 'fun', logy=True)